In [1]:
from sklearn.preprocessing import LabelEncoder
from bio import AlignIO
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio import SeqIO
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras_preprocessing import sequence
import sklearn
import numpy as np
import re

In [2]:
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^acgt0]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array

def readmaf(start, end, filename):
    count = 0
    oldlist =[]
    align = AlignIO.parse(filename, "maf")
    for multiple_alignment in align:
        count = count+1
        if count >start and count<=end:
            oldlist.append(multiple_alignment)
        elif count>end:
            break
    return oldlist

#oldlist stores MSAs
oldlist = readmaf(0, 1000000, "data/chr2.anc.maf")
print(oldlist[1][1])
print(dir(oldlist[1]))
print(oldlist[1][0].id)
#print(oldlist[2])

ID: panTro4.chr2A
Name: panTro4.chr2A
Number of features: 0
/start=4703
/size=1
/strand=1
/srcSize=113622374
Seq('a', SingleLetterAlphabet())
['__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_alphabet', '_annotations', '_append', '_get_per_column_annotations', '_per_col_annotations', '_records', '_set_per_column_annotations', '_str_line', 'add_sequence', 'annotations', 'append', 'column_annotations', 'extend', 'format', 'get_alignment_length', 'sort']
hg38.chr2


In [3]:
print(len(oldlist))

1000000


In [4]:
#alignList stores alignments between direct ancestor and descendent
def getAlign(inputList, keyword):
    alignList = []
    zeros = ''
    for i in range(len(inputList)):
        # index for the dog
        indexDes = -1
        indexAnc = -1

        for j in range(len(inputList[i])):
            if len(inputList[i][j])>=10 and 'canFam3' in inputList[i][j].id  :
                indexDes = j
            elif len(inputList[i][j])>=10 and inputList[i][j].id == '_CMAOL': #'_RMPC' :
                indexAnc = j
            '''    
            if j%1000000000 == 0 : 
                print(indexDes, indexAnc)'''
        if indexDes!=-1 and indexAnc!=-1:
            alignment = MultipleSeqAlignment([
                    SeqRecord(inputList[i][indexDes].seq+zeros, id=inputList[i][indexDes].id),
                    SeqRecord(inputList[i][indexAnc].seq+zeros, id=inputList[i][indexAnc].id),
                    #SeqRecord(zeros+inputList[i][indexDes].seq, id=inputList[i][indexDes].id)
                ]
            )
            alignList.append(alignment)
            '''
            if len(inputList[i][j])>=10 and j<len(inputList[i])-1 and (keyword[k] in inputList[i][j].id and keyword[k] in inputList[i][j+1].id):
                alignment = MultipleSeqAlignment([
                        SeqRecord(inputList[i][j].seq, id=inputList[i][j].id),
                        SeqRecord(inputList[i][j+1].seq, id=inputList[i][j+1].id)
                    ]
                )
                alignList.append(alignment)'''
    return alignList

def zeros(length):
    return "0"*length
    
FinalList = []    
anc = []
des = []
keyword = ["_HP","_RM","_FC","_CS","_BO","_PB","_TO","_VC", "_PP"]
alignList = getAlign(oldlist, keyword)
print(len(alignList))
print(alignList[126])
print(len(alignList[126][1].seq))
print(len(alignList[126][1]))
for i in range (100):
    print('printing new alignment -------------------------------------------\n')
    print(alignList[i])

#print(str(alignList[62][1]))
#print('The scikit-learn version is {}.'.format(sklearn.__version__))

357202
SingleLetterAlphabet() alignment with 2 rows and 20 columns
GATG---------------- canFam3.chr17
GATG---------------- _CMAOL
20
20
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 34 columns
AAGAGAG-AAACTAAAGCTGAGCCATAGGCTACA canFam3.chr17
AAGACAG-AAACTAAAGCTGAGCCATAGGCTACA _CMAOL
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 238 columns
A----------CTGACCTGAC----AGTCAGAGGAA-AGCAGGG...--- canFam3.chr17
A----------CTGATCTGACAATAAGTCAGGGGAA-AGAAGGG...--- _CMAOL
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 79 columns
------------------------------------------AG...TTT canFam3.chr17
------------------------------------------AG...TTT _CMAOL
printing new alignment -------------------------------------------

SingleLetterAlphabet() alignment with 2 rows and 14 columns
GCATTGTGTG---T

In [5]:
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','0', 'z']))
encoder = OneHotEncoder(sparse=False, dtype=int, categories=[range(6)])
def one_hot_encoder(my_array, encoder):
    integer_encoded = label_encoder.transform(my_array)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = encoder.fit_transform(integer_encoded)
    #onehot_encoded = np.delete(onehot_encoded, -1, 1)
    return onehot_encoded, encoder

def alignBoth(alignList):
    zeros = '00000'
    h, w = len(alignList), 2;
    Matrix = [[0 for x in range(w)] for y in range(h)]
    for i in range(len(alignList)):
        for j in range(len(alignList[i])):
            Matrix[i][j] = one_hot_encoder(string_to_array(str(alignList[i][j].seq).upper()), encoder)
    return Matrix

# [Index of alignment][1 for Ancester, 0 for Descendant][Always 0]
one_hot_aligned = alignBoth(alignList)
#one_hot_aligned = one_hot_aligned[:][:][0]
one_hot_aligned = np.array(one_hot_aligned)
#print(one_hot_aligned.shape)
#ignorethis, encoder = one_hot_encoder(string_to_array(str(alignList[1][1].seq)),encoder)


print(type(one_hot_aligned))
print(len(one_hot_aligned[126][1]))
print(one_hot_aligned[126][1][0])
#print(encoder.inverse_transform(one_hot_aligned[126][1]))
print(type(one_hot_aligned[126][1]))
# A = [1 0 0 0], C = [0 1 0 0], G = [0 0 1 0], T = [0 0 0 1]
def decoder(array):
    result = ""
    size = len(array)
    for i in range(size):
        if array[i].tolist() == [1, 0, 0, 0, 0, 0]:
            result=result+"0" 
        elif array[i].tolist() == [0, 1, 0, 0, 0, 0]:
            result=result+"A"
        elif array[i].tolist() == [0, 0, 1, 0, 0, 0]:
            result=result+"C"
        elif array[i].tolist() == [0, 0, 0, 1, 0, 0]:
            result=result+"G"
        elif array[i].tolist() == [0, 0, 0, 0, 1, 0]:
            result=result+"T"
        elif array[i].tolist() == [0, 0, 0, 0, 0, 1]:
            result=result+"-"
    return result

def decoderY(array):
    result = ""    
    if array.tolist() == [1, 0, 0, 0, 0, 0]:
        result=result+"0" 
    elif array.tolist() == [0, 1, 0, 0, 0, 0]:
        result=result+"A"
    elif array.tolist() == [0, 0, 1, 0, 0, 0]:
        result=result+"C"
    elif array.tolist() == [0, 0, 0, 1, 0, 0]:
        result=result+"G"
    elif array.tolist() == [0, 0, 0, 0, 1, 0]:
        result=result+"T"
    elif array.tolist() == [0, 0, 0, 0, 0, 1]:
        result=result+"-"
    return result
    
print(decoder(one_hot_aligned[126][1][0]))
print(len(one_hot_aligned))

<class 'numpy.ndarray'>
2
[[0 0 0 1 0 0]
 [0 1 0 0 0 0]
 [0 0 0 0 1 0]
 [0 0 0 1 0 0]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]
 [0 0 0 0 0 1]]
<class 'numpy.ndarray'>
GATG----------------
357202


In [6]:
def vectorize(char_setx, char_sety, sequence_length):
    dataX = []
    dataY = []
    dataX1 = []
    dataY1 = []
    result=[]
    #print(type(char_sety[0]))
    for i in range(len(char_setx) - sequence_length):
        
        '''for i in char_sety[i: i + sequence_length-1]:
            y1.append(i)'''
        #x1 = char_setx1[i: i + sequence_length]
        x = char_setx[i:i + sequence_length]
        y = char_sety[i: i + sequence_length]
        y1 = [[1, 0, 0, 0, 0, 0]]
        temp = y[:-1]
        temp = temp.tolist()
        for i in temp:
            y1.append(i)
        y1=np.array(y1)
        if mutation_with_nogap(x, y):# and mutation_with_nogap(x1, y1)
            dataX.append(x)
            dataY.append(y)
            dataY1.append(y1)

    return dataX, dataY, dataY1

def tokenize(one_hot_aligned, sequence_length):
    X = []
    Y = []
    X1 = []
    Y1 = []
    for i in range(len(one_hot_aligned)):
        dataX, dataY, dataY1 = vectorize(one_hot_aligned[i][1][0], one_hot_aligned[i][0][0],  sequence_length)
        X.extend(dataX)
        Y.extend(dataY)
        #X1.extend(dataX1)
        Y1.extend(dataY1)
    return X, Y, Y1

def truncate(one_hot_aligned):
    X = []
    Y = []
    for i in range(len(one_hot_aligned)):
        X.append(one_hot_aligned[i][1][0])
        Y.append(one_hot_aligned[i][0][0])
    X = sequence.pad_sequences(X,
                                 maxlen=30,
                                 truncating='post',
                                 padding='post',
                                 value=0)
    Y = sequence.pad_sequences(Y,
                                 maxlen=30,
                                 truncating='post',
                                 padding='post',
                                 value=0)
    return X,Y

# function for checking if there are at least one mutation and there are no gaps.
def mutation_with_nogap(a, b):
    a = a.tolist()
    b = b.tolist()
    if a.count([0,0,0,0,0,1])>0 or b.count([0,0,0,0,0,1])>0:
        return False
    else:
        return True
def diffList(a, b):
    count = 0
    length = len(a)
    for i in range(length):
        if a[i] != b[i]:
            count = count+1
    return count

def deleteGap(one_hot_aligned):
    result = one_hot_aligned
    for i in range(len(one_hot_aligned)):
        if one_hot_aligned[i][1][0] == '-' and one_hot_aligned[i][0][0] == '-':
            result.pop(i)
    return result

# Used for non 0 padding
one_hot_aligned = deleteGap(one_hot_aligned)
X, Y, Y1=tokenize(one_hot_aligned, 1)

# Used for 0 padding
#X, Y = truncate(one_hot_aligned)
X = np.array(X)
Y = np.array(Y)
#X1 = np.array(X1)
Y1 = np.array(Y1)

print(Y.shape)
print(len(X))
pt1 = 12000000
pt2 = 12100000
pt3 = 12290000
X_train = X[:pt1]
X_val = X[pt1:pt2]
X_test = X[pt2:pt3]
y_train = Y[:pt1]
y_val =  Y[pt1:pt2]
y_test = Y[pt2:pt3]
# X_train1 = X1[:pt1]
# X_val1 = X1[pt1:pt2]
# X_test1 = X1[pt2:pt3]
y_train1 = Y1[:pt1]
y_val1 =  Y1[pt1:pt2]
y_test1 = Y1[pt2:pt3]

# X_train = X[:20000]
# X_val = X[20000:23000]
# X_test = X[23000:26000]
# y_train = Y[:20000]
# y_val =  Y[20000:23000]
# y_test = Y[23000:26000]

np.save('prepData/X_train_camFam3_v3_chr2_size1', X_train)
np.save('prepData/X_val_camFam3_v3_chr2_size1', X_val)
np.save('prepData/X_test_camFam3_v3_chr2_size1', X_test)
np.save('prepData/y_train_camFam3_v3_chr2_size1', y_train)
np.save('prepData/y_val_camFam3_v3_chr2_size1', y_val)
np.save('prepData/y_test_camFam3_v3_chr2_size1', y_test)

np.save('prepData/y_train1_camFam3_v3_chr2_size1', y_train1)
np.save('prepData/y_val1_camFam3_v3_chr2_size1', y_val1)
np.save('prepData/y_test1_camFam3_v3_chr2_size1', y_test1)

# np.save('prepData/X_train1_camFam3_1mutOnly_v2', X_train1)
# np.save('prepData/X_val1_camFam3_1mutOnly_v2', X_val1)
# np.save('prepData/X_test1_camFam3_1mutOnly_v2', X_test1)


# np.save('prepData20/X_train_camFam3_1mutOnly', X_train)
# np.save('prepData20/X_val_camFam3_1mutOnly', X_val)
# np.save('prepData20/X_test_camFam3_1mutOnly', X_test)
# np.save('prepData20/y_train_camFam3_1mutOnly', y_train)
# np.save('prepData20/y_val_camFam3_1mutOnly', y_val)
# np.save('prepData20/y_test_camFam3_1mutOnly', y_test)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(X_train[0])
print(y_train[0])
print(y_train1[0])


/home/mcb/dlim63/miniconda3/envs/research/lib/python3.6/site-packages/ipykernel_launcher.py:78: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison


(12292970, 1, 6)
12292970
(12000000, 1, 6)
(100000, 1, 6)
(190000, 1, 6)
[[0 1 0 0 0 0]]
[[0 1 0 0 0 0]]
[[1 0 0 0 0 0]]


In [8]:
# import difflib
# sm=difflib.SequenceMatcher(None,X_test,X_train)
# sm2=difflib.SequenceMatcher(None,y_test,y_train)
for i in range(len(X)):
    print(i, decoder(X[i]))

0 A
1 A
2 G
3 A
4 C
5 A
6 G
7 A
8 A
9 A
10 C
11 T
12 A
13 A
14 A
15 G
16 C
17 T
18 G
19 A
20 G
21 C
22 C
23 A
24 T
25 A
26 G
27 G
28 C
29 T
30 A
31 C
32 A
33 C
34 T
35 G
36 A
37 T
38 C
39 T
40 G
41 A
42 C
43 A
44 G
45 T
46 C
47 A
48 G
49 G
50 G
51 G
52 A
53 A
54 A
55 G
56 A
57 A
58 G
59 G
60 G
61 C
62 A
63 C
64 A
65 G
66 A
67 G
68 A
69 G
70 A
71 T
72 T
73 T
74 G
75 A
76 A
77 C
78 A
79 T
80 T
81 T
82 G
83 T
84 C
85 C
86 A
87 C
88 T
89 A
90 A
91 A
92 A
93 A
94 A
95 A
96 T
97 G
98 C
99 T
100 A
101 G
102 A
103 A
104 T
105 T
106 C
107 A
108 T
109 A
110 A
111 A
112 A
113 T
114 T
115 G
116 A
117 T
118 T
119 C
120 C
121 A
122 A
123 G
124 T
125 A
126 A
127 A
128 T
129 A
130 A
131 T
132 A
133 A
134 T
135 A
136 A
137 T
138 C
139 A
140 A
141 T
142 A
143 A
144 C
145 A
146 A
147 A
148 G
149 G
150 A
151 T
152 C
153 T
154 A
155 C
156 A
157 G
158 T
159 A
160 G
161 G
162 A
163 A
164 C
165 T
166 G
167 A
168 A
169 C
170 A
171 G
172 A
173 T
174 C
175 T
176 C
177 A
178 C
179 A
180 A
181 A
182 T
183 G
184 G


1464 C
1465 A
1466 C
1467 C
1468 A
1469 C
1470 C
1471 T
1472 G
1473 C
1474 A
1475 C
1476 T
1477 A
1478 G
1479 G
1480 T
1481 G
1482 G
1483 C
1484 A
1485 C
1486 C
1487 T
1488 A
1489 G
1490 A
1491 A
1492 G
1493 G
1494 T
1495 T
1496 C
1497 A
1498 A
1499 T
1500 T
1501 A
1502 A
1503 A
1504 T
1505 A
1506 C
1507 C
1508 A
1509 A
1510 A
1511 A
1512 A
1513 C
1514 A
1515 A
1516 A
1517 A
1518 C
1519 A
1520 A
1521 A
1522 A
1523 A
1524 A
1525 T
1526 G
1527 T
1528 G
1529 C
1530 T
1531 C
1532 C
1533 A
1534 C
1535 T
1536 A
1537 C
1538 T
1539 T
1540 T
1541 T
1542 G
1543 A
1544 A
1545 A
1546 T
1547 C
1548 T
1549 A
1550 C
1551 T
1552 G
1553 T
1554 T
1555 A
1556 T
1557 C
1558 T
1559 A
1560 A
1561 T
1562 T
1563 T
1564 T
1565 T
1566 A
1567 A
1568 C
1569 T
1570 G
1571 T
1572 T
1573 T
1574 A
1575 A
1576 A
1577 T
1578 G
1579 A
1580 A
1581 T
1582 A
1583 A
1584 A
1585 C
1586 T
1587 T
1588 C
1589 C
1590 T
1591 G
1592 T
1593 T
1594 C
1595 A
1596 T
1597 C
1598 T
1599 T
1600 G
1601 G
1602 C
1603 A
1604 A
1605 G
1606 A

2963 G
2964 A
2965 G
2966 C
2967 A
2968 T
2969 T
2970 T
2971 A
2972 T
2973 T
2974 T
2975 A
2976 T
2977 G
2978 A
2979 G
2980 T
2981 T
2982 A
2983 G
2984 T
2985 T
2986 A
2987 T
2988 A
2989 T
2990 A
2991 T
2992 G
2993 A
2994 C
2995 T
2996 G
2997 A
2998 G
2999 C
3000 T
3001 G
3002 A
3003 T
3004 G
3005 T
3006 G
3007 G
3008 T
3009 T
3010 A
3011 T
3012 A
3013 T
3014 G
3015 T
3016 T
3017 T
3018 A
3019 C
3020 A
3021 T
3022 G
3023 T
3024 T
3025 T
3026 A
3027 T
3028 T
3029 C
3030 C
3031 A
3032 A
3033 A
3034 T
3035 A
3036 G
3037 A
3038 G
3039 T
3040 A
3041 A
3042 G
3043 A
3044 C
3045 A
3046 C
3047 A
3048 G
3049 G
3050 A
3051 A
3052 A
3053 A
3054 A
3055 G
3056 T
3057 G
3058 T
3059 G
3060 C
3061 G
3062 G
3063 G
3064 C
3065 T
3066 T
3067 T
3068 G
3069 A
3070 G
3071 A
3072 C
3073 G
3074 A
3075 G
3076 A
3077 A
3078 G
3079 G
3080 A
3081 G
3082 C
3083 C
3084 A
3085 G
3086 C
3087 C
3088 T
3089 G
3090 C
3091 A
3092 T
3093 C
3094 C
3095 A
3096 C
3097 G
3098 T
3099 T
3100 C
3101 C
3102 C
3103 C
3104 C
3105 T

4401 G
4402 A
4403 C
4404 A
4405 C
4406 T
4407 C
4408 T
4409 G
4410 T
4411 G
4412 G
4413 T
4414 A
4415 T
4416 C
4417 C
4418 A
4419 C
4420 A
4421 G
4422 A
4423 A
4424 G
4425 T
4426 G
4427 G
4428 G
4429 A
4430 G
4431 T
4432 G
4433 C
4434 T
4435 A
4436 A
4437 C
4438 C
4439 C
4440 C
4441 A
4442 C
4443 A
4444 A
4445 T
4446 T
4447 T
4448 C
4449 A
4450 C
4451 C
4452 T
4453 G
4454 G
4455 A
4456 A
4457 A
4458 T
4459 T
4460 A
4461 G
4462 C
4463 A
4464 G
4465 A
4466 T
4467 T
4468 A
4469 A
4470 A
4471 G
4472 A
4473 G
4474 C
4475 A
4476 A
4477 G
4478 G
4479 G
4480 C
4481 C
4482 A
4483 C
4484 T
4485 T
4486 G
4487 A
4488 A
4489 C
4490 T
4491 C
4492 T
4493 T
4494 G
4495 A
4496 G
4497 G
4498 T
4499 C
4500 C
4501 T
4502 T
4503 T
4504 A
4505 T
4506 T
4507 T
4508 C
4509 C
4510 T
4511 G
4512 G
4513 T
4514 T
4515 G
4516 G
4517 C
4518 A
4519 A
4520 C
4521 C
4522 T
4523 G
4524 C
4525 A
4526 C
4527 C
4528 A
4529 T
4530 G
4531 G
4532 C
4533 C
4534 C
4535 C
4536 T
4537 G
4538 A
4539 G
4540 G
4541 C
4542 A
4543 C

KeyboardInterrupt: 

In [ ]:
for i in range(len(Y)):
    print(i, decoder(Y[i]))

In [ ]:
for i in range(len(Y)):
    print(i, decoder(Y1[i]))